In [65]:
 # Dependencies and Setup
import pandas as pd

In [66]:
# Files to Load
student_input = "Resources/students_complete.csv"
school_input = "Resources/schools_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
student_data = pd.read_csv(student_input)
school_data = pd.read_csv(school_input)

# Combine the data into a single dataset.  
df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


## District Summary

**Create a high-level snapshot, in a DataFrame, of the district's key metrics, including the following:**

- Total schools
- Total students
- Total budget
- Average math score
- Average reading score
- % passing math (the percentage of students who passed math)
- % passing reading (the percentage of students who passed reading)
- % overall passing (the percentage of students who passed math AND reading)

In [67]:
# calculate totals for schools and students
total_schools = len(df["school_name"].unique())
total_students = df["Student ID"].count()

# calculate the total budget
total_budget = school_data["budget"].sum()

In [68]:
# calculate average scores
avg_math = df["math_score"].mean()
avg_reading = df["reading_score"].mean()

In [69]:
# calculate % passing for math
pass_math = df.loc[(df["math_score"] >= 70.0)].math_score.count()
percent_pass_math = (pass_math / total_students) * 100

In [70]:
# calculate % passing for reading
pass_reading = df.loc[(df["reading_score"] >= 70.0)].reading_score.count()
percent_pass_reading = (pass_reading / total_students) * 100

In [71]:
# calculate % passing for both
pass_both = df.loc[(df["math_score"] >= 70.0) & (df["reading_score"] >= 70.0)].reading_score.count()
percent_pass_both = (pass_both / total_students) * 100

In [72]:
district_df = pd.DataFrame(columns=('Total Schools','Total Students','Total Budget','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing'))

district_df.loc[0] = [total_schools,total_students,total_budget,avg_math,avg_reading,percent_pass_math,percent_pass_reading,percent_pass_both]

district_df["Total Schools"] = district_df["Total Schools"].astype(int)
district_df["Total Students"] = district_df["Total Students"].map("{:,.0f}".format)
district_df["Total Budget"] = district_df["Total Budget"].map("${:,.2f}".format)
district_df["% Passing Math"] = district_df["% Passing Math"].map("{:.5f}%".format)
district_df["% Passing Reading"] = district_df["% Passing Reading"].map("{:.5f}%".format)
district_df["% Overall Passing"] = district_df["% Overall Passing"].map("{:.5f}%".format)

district_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.98085%,85.80546%,65.17233%


## School Summary

**Create a DataFrame that summarizes key metrics about each school, including the following:**

- School name
- School type
- Total students
- Total school budget
- Per student budget
- Average math score
- Average reading score
- % passing math (the percentage of students who passed math)
- % passing reading (the percentage of students who passed reading)
- % overall passing (the percentage of students who passed math AND reading)

In [73]:
# group by school / school name is index
# used groupby / agg to count student names and find avg for math and reading scores
studentcount_avgs_df = df.set_index(["school_name"]).sort_index()
studentcount_avgs_df = studentcount_avgs_df.groupby("school_name").agg({'student_name': 'count', 'math_score': 'mean', 'reading_score': 'mean'})
studentcount_avgs_df = studentcount_avgs_df.rename(columns={"student_name":"Total Students", "math_score":"Average Math Score", "reading_score":"Average Reading Score"})

In [74]:
# create df with school name as index, with school type and budget
grouped_school_data_df = school_data.set_index(["school_name"]).sort_index()
grouped_school_data_df = grouped_school_data_df[(["type", "budget"])]
grouped_school_data_df = grouped_school_data_df.rename(columns={"type":"School Type", "budget": "Total School Budget"})

# merge this data pulled from school_data w/ other merged dataframes
merged_dfs = pd.merge(grouped_school_data_df,studentcount_avgs_df, on="school_name")


In [75]:
# create mask to only include data with students who passed math
passed_math_df = df.loc[(df["math_score"] >= 70.0)]
# create df with school name as index, with count of test scores 70 or higher in math
math_grouped_df = passed_math_df.groupby("school_name").agg({'math_score': 'count'})
math_grouped_df = math_grouped_df.rename(columns={"math_score":"count_passed_math"})

# create mask to only include data with students who passed reading
passed_reading_df = df.loc[(df["reading_score"] >= 70.0)]
# create df with school name as index, with count of test scores 70 or higher in reading
reading_grouped_df = passed_reading_df.groupby("school_name").agg({'reading_score': 'count'})
reading_grouped_df = reading_grouped_df.rename(columns={"reading_score":"count_passed_reading"})

# create mask to only include data with students who passed both math & reading
passed_both_df = df.loc[(df["math_score"] >= 70.0) & (df["reading_score"] >= 70.0)]
# create df with school name as index, with count of test scores 70 or higher in both math & reading
both_grouped_df = passed_both_df.groupby("school_name").agg({'reading_score': 'count'})
both_grouped_df = both_grouped_df.rename(columns={"reading_score":"count_passed_overall"})


In [76]:
# merge dataframes created so far
merged_dfs = pd.merge(merged_dfs, math_grouped_df,on="school_name").join(reading_grouped_df, on="school_name").join(both_grouped_df, on="school_name")
#merged_dfs = pd.merge(merged_dfs,reading_grouped_df, on="school_name")
#merged_dfs = pd.merge(merged_dfs,both_grouped_df, on="school_name")

# reorder
merged_dfs = merged_dfs[["School Type", "Total Students", "Total School Budget", "Average Math Score","Average Reading Score", "count_passed_math", "count_passed_reading", "count_passed_overall"]]

In [77]:
# add new column, calculating per student budget
merged_dfs["Per Student Budget"] = merged_dfs["Total School Budget"] / merged_dfs["Total Students"]

# add new column, calculating % passed math
merged_dfs["% Passing Math"] = (merged_dfs["count_passed_math"] / merged_dfs["Total Students"]) * 100

# add new column, calculating % passed reading
merged_dfs["% Passing Reading"] = (merged_dfs["count_passed_reading"] / merged_dfs["Total Students"]) * 100

# add new column, calculating % passed math & reading
merged_dfs["% Overall Passing"] = (merged_dfs["count_passed_overall"] / merged_dfs["Total Students"]) * 100

In [78]:
# remove extra columns
school_summary_df = merged_dfs.drop(columns=["count_passed_math","count_passed_reading","count_passed_overall"])

# rearrange columns
formatted_school_summary_df = school_summary_df[["School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score","Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]

In [79]:
# format columns
formatted_school_summary_df["Total Students"] = formatted_school_summary_df["Total Students"].map("{:,.0f}".format)
formatted_school_summary_df["Total School Budget"] = formatted_school_summary_df["Total School Budget"].map("${:,.2f}".format)
formatted_school_summary_df["Per Student Budget"] = formatted_school_summary_df["Per Student Budget"].map("${:,.2f}".format)
formatted_school_summary_df["% Passing Math"] = formatted_school_summary_df["% Passing Math"].map("{:.5f}%".format)
formatted_school_summary_df["% Passing Reading"] = formatted_school_summary_df["% Passing Reading"].map("{:.5f}%".format)
formatted_school_summary_df["% Overall Passing"] = formatted_school_summary_df["% Overall Passing"].map("{:.5f}%".format)

formatted_school_summary_df.index.name = None

In [80]:
formatted_school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.68006%,81.93328%,54.64228%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.13348%,97.03983%,91.33477%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.98847%,80.73923%,53.20448%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.30960%,79.29901%,54.28989%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.39237%,97.13896%,90.59946%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.75297%,80.86300%,53.52751%
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.50585%,96.25293%,89.22717%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.68392%,81.31642%,53.51388%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.05755%,81.22243%,53.53917%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.59459%,95.94595%,90.54054%


## Highest-Performing Schools (by % Overall Passing)

Create a DataFrame that highlights the top 5 performing schools based on % Overall Passing. Include the following metrics:

* School name
* School type
* Total students
* Total school budget
* Per student budget
* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

In [81]:
#high_sorted_school_summary_df = school_summary_df.sort_values('% Overall Passing', ascending=False).head(5)
high_sorted_school_summary_df = school_summary_df.sort_values('% Overall Passing', ascending=False).iloc[:5,:]

high_sorted_school_summary_df.head(10)

,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,1081356,83.061895,83.975780,582.0,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,1043130,83.418349,83.848930,638.0,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,917500,83.351499,83.816757,625.0,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574,83.274201,83.989488,578.0,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858,83.839917,84.044699,609.0,94.594595,95.945946,90.540541


## Lowest-Performing Schools (by % Overall Passing)

Create a DataFrame that highlights the bottom 5 performing schools based on % Overall Passing. Include the following metrics:

* School name
* School type
* Total students
* Total school budget
* Per student budget
* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

In [82]:
low_sorted_school_summary_df = school_summary_df.sort_values('% Overall Passing').iloc[:5,:]
low_sorted_school_summary_df

,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,2547363,76.842711,80.744686,637.0,66.366592,80.220055,52.988247
Figueroa High School,District,2949,1884411,76.711767,81.158020,639.0,65.988471,80.739234,53.204476
Huang High School,District,2917,1910635,76.629414,81.182722,655.0,65.683922,81.316421,53.513884
Hernandez High School,District,4635,3022020,77.289752,80.934412,652.0,66.752967,80.862999,53.527508
Johnson High School,District,4761,3094650,77.072464,80.966394,650.0,66.057551,81.222432,53.539172


## Math Scores by Grade

Create a DataFrame that lists the average math score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [83]:
#math_bygrade_df = df.groupby(["school_name","grade"]).agg({"math_score":"mean"})
#math_bygrade_df = math_bygrade_df.rename(columns={"math_score": "Average Math Score"})
#math_bygrade_df

In [84]:
#create df for each grade level
math_mask9th = df[df.grade == "9th"].groupby("school_name").agg({"math_score":"mean"}).rename(columns={"math_score": "9th"})
math_mask10th = df[df.grade == "10th"].groupby("school_name").agg({"math_score":"mean"}).rename(columns={"math_score": "10th"})
math_mask11th = df[df.grade == "11th"].groupby("school_name").agg({"math_score":"mean"}).rename(columns={"math_score": "11th"})
math_mask12th = df[df.grade == "12th"].groupby("school_name").agg({"math_score":"mean"}).rename(columns={"math_score": "12th"})

#merge grade levels data into one df
math_merged_grades = pd.merge(math_mask9th,math_mask10th, on="school_name").join(math_mask11th,on="school_name").join(math_mask12th,on="school_name")
math_merged_grades.index.name = None
math_merged_grades

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Scores by Grade

Create a DataFrame that lists the average reading score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [85]:
#reading_bygrade_df = df.groupby(["school_name","grade"]).agg({"reading_score":"mean"})
#reading_bygrade_df = reading_bygrade_df.rename(columns={"reading_score": "Average Reading Score"})
#reading_bygrade_df

In [86]:
#create df for each grade level
reading_math_mask9th = df[df.grade == "9th"].groupby("school_name").agg({"reading_score":"mean"}).rename(columns={"reading_score": "9th"})
reading_math_mask10th = df[df.grade == "10th"].groupby("school_name").agg({"reading_score":"mean"}).rename(columns={"reading_score": "10th"})
reading_math_mask11th = df[df.grade == "11th"].groupby("school_name").agg({"reading_score":"mean"}).rename(columns={"reading_score": "11th"})
reading_math_mask12th = df[df.grade == "12th"].groupby("school_name").agg({"reading_score":"mean"}).rename(columns={"reading_score": "12th"})

#merge grade levels data into one df
reading_merged_grades = pd.merge(reading_math_mask9th,reading_math_mask10th, on="school_name").join(reading_math_mask11th,on="school_name").join(reading_math_mask12th,on="school_name")
reading_merged_grades.index.name = None
reading_merged_grades

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

Create a table that breaks down school performance based on average spending ranges (per student). Use your judgment to create four bins with reasonable cutoff values to group school spending. Include the following metrics in the table:

* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

In [102]:
# using df created above as starting point, setting with new name
scores_by_spending_df = school_summary_df[["School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score","Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]



In [103]:
# create bins for values
bins = 0, 585, 630, 645, 680
# create labels for bins
labels = "<$585", "$585-630", "$630-645", "$645-680"

In [104]:
# Slice the data and place it into bins
scores_by_spending_df["Spending Ranges (Per Student)"] = pd.cut(scores_by_spending_df["Per Student Budget"], bins, labels=labels,include_lowest=True)

/var/folders/q2/5v2xmgf12v118bg1jrms5ph80000gn/T/ipykernel_23545/2650669811.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scores_df["Spending Ranges (Per Student)"] = pd.cut(scores_df["Per Student Budget"], bins, labels=labels,include_lowest=True)


In [105]:
# resetting index to be the new bin labels, sorting low to high
scores_by_spending_df = scores_by_spending_df.set_index(["Spending Ranges (Per Student)"]).sort_index()
# calculating avg of amounts within each bin
scores_by_spending_df = scores_by_spending_df.groupby("Spending Ranges (Per Student)").agg({'Average Math Score': 'mean', 'Average Reading Score': 'mean', '% Passing Math':'mean', '% Passing Reading': 'mean','% Overall Passing': 'mean'})


In [106]:
# formatting to 2 places after the decimal
scores_by_spending_df["Average Math Score"] = scores_by_spending_df["Average Math Score"].map("{:,.2f}".format)
scores_by_spending_df["Average Reading Score"] = scores_by_spending_df["Average Reading Score"].map("{:,.2f}".format)
scores_by_spending_df["% Passing Math"] = scores_by_spending_df["% Passing Math"].map("{:,.2f}".format)
scores_by_spending_df["% Passing Reading"] = scores_by_spending_df["% Passing Reading"].map("{:,.2f}".format)
scores_by_spending_df["% Overall Passing"] = scores_by_spending_df["% Overall Passing"].map("{:,.2f}".format)

scores_by_spending_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

Create a table that breaks down school performance based on school size (small, medium, or large).

In [128]:
# using df created above as starting point, setting with new name
scores_by_size_df = school_summary_df[["Total Students", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]



In [129]:
# create bins for values
bins = 0, 1000, 2000, 5000
# create labels for bins
labels = "Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"

In [130]:
# Slice the data and place it into bins
scores_by_size_df["School Size"] = pd.cut(scores_by_size_df["Total Students"], bins, labels=labels,include_lowest=True)

/var/folders/q2/5v2xmgf12v118bg1jrms5ph80000gn/T/ipykernel_23545/1214359052.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scores_by_size_df["School Size"] = pd.cut(scores_by_size_df["Total Students"], bins, labels=labels,include_lowest=True)


In [131]:
# resetting index to be the new bin labels, sorting descending
scores_by_size_df = scores_by_size_df.set_index(["School Size"]).sort_index()
# calculating avg of amounts within each bin
scores_by_size_df = scores_by_size_df.groupby("School Size").agg({'Average Math Score': 'mean', 'Average Reading Score': 'mean', '% Passing Math':'mean', '% Passing Reading': 'mean','% Overall Passing': 'mean'})

In [132]:
# formatting to 2 places after the decimal
scores_by_size_df["Average Math Score"] = scores_by_size_df["Average Math Score"].map("{:,.2f}".format)
scores_by_size_df["Average Reading Score"] = scores_by_size_df["Average Reading Score"].map("{:,.2f}".format)
scores_by_size_df["% Passing Math"] = scores_by_size_df["% Passing Math"].map("{:,.2f}".format)
scores_by_size_df["% Passing Reading"] = scores_by_size_df["% Passing Reading"].map("{:,.2f}".format)
scores_by_size_df["% Overall Passing"] = scores_by_size_df["% Overall Passing"].map("{:,.2f}".format)

scores_by_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.86,93.60,96.79,90.62
Large (2000-5000),77.75,81.34,69.96,82.77,58.29


## Scores by School Type

Create a table that breaks down school performance based on type of school (district or charter).

In [141]:
# using df created above as starting point, setting with new name
scores_by_type_df = school_summary_df[["School Type", "Total Students", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]


In [142]:
# resetting index to be the new bin labels, sorting descending
scores_by_type_df = scores_by_type_df.set_index(["School Type"]).sort_index()
# calculating avg of amounts within each bin
scores_by_type_df = scores_by_type_df.groupby("School Type").agg({'Average Math Score': 'mean', 'Average Reading Score': 'mean', '% Passing Math':'mean', '% Passing Reading': 'mean','% Overall Passing': 'mean'})

In [143]:
# formatting to 2 places after the decimal
scores_by_type_df["Average Math Score"] = scores_by_type_df["Average Math Score"].map("{:,.2f}".format)
scores_by_type_df["Average Reading Score"] = scores_by_type_df["Average Reading Score"].map("{:,.2f}".format)
scores_by_type_df["% Passing Math"] = scores_by_type_df["% Passing Math"].map("{:,.2f}".format)
scores_by_type_df["% Passing Reading"] = scores_by_type_df["% Passing Reading"].map("{:,.2f}".format)
scores_by_type_df["% Overall Passing"] = scores_by_type_df["% Overall Passing"].map("{:,.2f}".format)

scores_by_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
